### Tools In Langchain
1. How To Create Tools
2. How to use built-in tools and toolkits
3. How to use chat models to call tools
4. How to pass tool outputs to chat models



This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [2]:
## @tool decorator

from langchain_core.tools import tool

@tool
def division(a:int,b:int)->int:
    """Divide 2 number"""
    return a/b


In [4]:
print(division.name)
print(division.description)
print(division.args)

division
Divide 2 number
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
## async implementation

from langchain_core.tools import tool


@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [6]:
from typing import Annotated,List

@tool
def multiply_by_max(
    a: Annotated[int,"A value"],
    b:Annotated[List[int],"list of ints over which to take maximum"]
)->int:
    """Mulitply a by the maximum of b"""
    return a* max(b)

In [7]:
print(multiply_by_max.args)

{'a': {'description': 'A value', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}


### Structured Tool
The StructuredTool.from_function class method provides a bit more configurability than the @tool decorator, without requiring much additional code.

In [8]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator=StructuredTool.from_function(func=multiply,coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

6
10


### Inbuilt Tools

### wikipedia Integration

In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query":"langchain"}))


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors


In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [20]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [21]:
tool.invoke("What is the recent ai news")

{'query': 'What is the recent ai news',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=ASWzDAhmJ_GQNq6W7fXWQIPczJ5rpf5czz9H4GBpXDhYX0Mm-TgaWUTHHwxB&gaa_ts=68a30010&gaa_sig=DmHwGbGlTN-xvBYj0IKljr_92NVR9QP43B5YAewKNmW2g1n347BGkcFB4Y2XC9AgF0CtkYlxJnpB-_KsiHY-tg%3D%3D',
   'title': 'Artificial Intelligence - Latest AI News and Analysis - WSJ.com',
   'content': 'The latest artificial intelligence news coverage focusing on the technology, tools and the companies building AI technology.',
   'score': 0.67758125,
   'raw_content': None},
  {'url': 'https://www.crescendo.ai/news/latest-ai-news-and-updates',
   'title': 'Latest AI Breakthroughs and News: June, July, August 2025',
   'content': 'AI Boosts Early Detection of Diabetic Eye Disease ... Summary: New research shows AI can accurately screen for diabetic retinopathy—a leading cause of vision loss',
   'score': 0.67527276,
   'raw_content': None},
  {'url'

In [23]:
from langchain_community.utilities import ArxivAPIWrapper

tool=ArxivAPIWrapper()

tool.run("1706.03762")

e:\AgenticAIHindi\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:102: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()
e:\AgenticAIHindi\.venv\Lib\site-packages\feedparser\html.py:152: DeprecationWarning: 'count' is passed as positional argument
  data = re.sub(r'<!((?!DOCTYPE|--|\[))', r'&lt;!\1', data, re.IGNORECASE)
e:\AgenticAIHindi\.venv\Lib\site-packages\feedparser\html.py:152: DeprecationWarning: 'count' is passed as positional argument
  data = re.sub(r'<!((?!DOCTYPE|--|\[))', r'&lt;!\1', data, re.IGNORECASE)


'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntranslation task, our model establis

### Call Tool With LLM Model

In [26]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(wiki_tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [29]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [27]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

In [38]:
wiki_tool.name

'wikipedia'

In [39]:
add.name

'add'

In [40]:
multiply.name

'multiply'

In [41]:
tavily_tool.name

'tavily_search'

In [30]:
tools=[wiki_tool,add,multiply,tavily_tool]

In [31]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\AgenticAIHindi\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100)),
 StructuredTool(name='add', description='Adds a and b.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000285E792D3A0>),
 StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x00000285E786C4A0>),
 TavilySearch(max_results=5, topic='general', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

In [32]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("qwen/qwen3-32b",model_provider="groq")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000285980F3620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000285982782F0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [33]:
llm_with_tool=llm.bind_tools(tools)
llm_with_tool

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000285980F3620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000285982782F0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'query to look up on wikipedia', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'add', 'description': 'Adds a and b.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'descripti

In [44]:
from langchain_core.messages import HumanMessage
query="What is 2* 3"
messages = [HumanMessage(query)]
response=llm_with_tool.invoke(query)
print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 2* 3". Let me see. The multiplication symbol is a bit unclear, but I think they mean 2 multiplied by 3. Let me check the available tools. There\'s a multiply function that takes two integers. So I should call multiply with a=2 and b=3. That should give the correct result of 6. I don\'t need to use any other tools here since it\'s a straightforward multiplication.\n', 'tool_calls': [{'id': 'dbrg7ya4r', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 2058, 'total_tokens': 2184, 'completion_time': 0.253766971, 'prompt_time': 0.12005249, 'queue_time': 0.060788129, 'total_time': 0.373819461}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b38e54eb-77cf-474c-b4ec-78eb485730f0-0' tool_calls=[{'name': 'multiply', '

In [35]:
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user asked "What is 2* 3". First, I need to figure out what they\'re asking. The asterisk (*) is a multiplication symbol, so they want to know the product of 2 and 3. \n\nLooking at the tools provided, there\'s a function called "multiply" which takes two integers as parameters. The description says it multiplies a and b. That\'s exactly what I need here.\n\nSo, the parameters required are "a" and "b", both integers. The user\'s input is 2 and 3. I should call the multiply function with a=2 and b=3. \n\nI need to make sure there are no other functions that could be used here. The "add" function is for addition, which isn\'t needed here. The other functions like wikipedia and tavily_search are for more complex queries, not simple arithmetic. \n\nTherefore, the correct tool to use is the multiply function with the given parameters. The result should be 6.\n', 'tool_calls': [{'id': '6qeka5s6c', 'function'

In [36]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '6qeka5s6c',
  'type': 'tool_call'}]

In [46]:

for tool_call in response.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages


[HumanMessage(content='What is 2* 3', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='6', name='multiply', tool_call_id='dbrg7ya4r')]

In [43]:
messages

[ToolMessage(content='6', name='multiply', tool_call_id='6qeka5s6c')]

In [47]:
llm_with_tool.invoke(messages)

AIMessage(content='6', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 2* 3". Let me see. The tools available include a multiply function. The parameters for multiply are a and b, both integers. So 2 multiplied by 3 would be 6. The answer is straightforward. I should call the multiply function with a=2 and b=3.\n'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 2068, 'total_tokens': 2144, 'completion_time': 0.149768476, 'prompt_time': 0.124041511, 'queue_time': 0.061685618, 'total_time': 0.273809987}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3a09688-18dd-4a6a-8ec6-2b71d8c1493d-0', usage_metadata={'input_tokens': 2068, 'output_tokens': 76, 'total_tokens': 2144})

In [48]:
from langchain_core.messages import HumanMessage

query = "What is langchain and what is 5*15?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking two things here: what LangChain is and the result of 5 multiplied by 15. Let me break this down.\n\nFirst, for the definition of LangChain, I need to use the Wikipedia tool. The user might want a general explanation, so I\'ll look up "LangChain" on Wikipedia. That should give an overview of what it is, its purpose, and maybe some key features. I should make sure to get the most accurate and up-to-date information from the Wikipedia entry.\n\nNext, the math problem: 5 times 15. This is straightforward arithmetic. The multiply function is available, so I can use that. The parameters are a=5 and b=15. The function should return 75. I need to call both functions and present the answers clearly.\n\nWait, the user might expect a quick answer for the multiplication, so using the multiply tool makes sense. For LangChain, since it\'s a concept, Wikipedia is the right choice. Let me check if there are any oth

In [49]:
ai_msg.tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'LangChain'},
  'id': 'pypqfsacv',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 5, 'b': 15},
  'id': '1bd2kyw6k',
  'type': 'tool_call'}]

In [50]:

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is langchain and what is 5*15?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of ', name='wikipedia', tool_call_id='pypqfsacv'),
 ToolMessage(content='75', name='multiply', tool_call_id='1bd2kyw6k')]

In [52]:
response=llm_with_tool.invoke(messages)

In [54]:
response.content

'LangChain is a software framework designed to simplify the integration of language models and other AI components into applications. It provides tools for building workflows, managing data, and creating interactive systems like chatbots or analysis tools. \n\nAdditionally, $5 \\times 15 = 75$.'

In [57]:
from langchain_core.messages import HumanMessage

query = "What is langchain and what is 5*15 and summarize the recent AI news ?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tool.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let\'s break down the user\'s query. They asked three things: what is LangChain, calculate 5*15, and summarize recent AI news. \n\nFirst, for LangChain, I need to explain what it is. Since it\'s a technical tool, maybe Wikipedia has a good overview. I should use the wikipedia function with "LangChain" as the query.\n\nNext, 5*15 is a math problem. The multiply function is perfect here. The parameters are a=5 and b=15. That should give the product, which is 75.\n\nLastly, summarizing recent AI news requires current information. The tavily_search function can fetch recent articles. I\'ll set the query to "recent AI news", set the time_range to "week" to get the latest updates, and maybe include some domains like tech news sites. But since the user didn\'t specify domains, I\'ll leave include_domains as default. Search depth should be advanced to ensure comprehensive results.\n\nWait, do I need to exclude any domains? Th

In [58]:
ai_msg.tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'LangChain'},
  'id': 'qj7snxn3h',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 5, 'b': 15},
  'id': 'nrfc9khvz',
  'type': 'tool_call'},
 {'name': 'tavily_search',
  'args': {'query': 'recent AI news',
   'search_depth': 'advanced',
   'time_range': 'week'},
  'id': '0t10pegj3',
  'type': 'tool_call'}]

In [59]:

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply,"wikipedia":wiki_tool,"tavily_search":tavily_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content='What is langchain and what is 5*15 and summarize the recent AI news ?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of ', name='wikipedia', tool_call_id='qj7snxn3h'),
 ToolMessage(content='75', name='multiply', tool_call_id='nrfc9khvz'),
 ToolMessage(content='{"query": "recent AI news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://champaignmagazine.com/2025/08/17/ai-by-ai-weekly-top-5-august-11-17-2025/", "title": "AI by AI Weekly Top 5 (August 11 – 17, 2025) - Champaign Magazine", "content": "Date: August 17, 2025  \\nContext: Google’s Gemini AI introduced a new “Projects” feature that helps users organize long-term research tasks, similar to ChatGPT’s project mode.  \\nSignificance: This marks a shift toward AI as a productivity partner, enabling deeper research, better context retention, and more st

In [60]:
response=llm_with_tool.invoke(messages)
response.content

'1. **LangChain**: According to Wikipedia, *LangChain* is a software framework designed to facilitate the integration of language models and other AI components into applications. It provides tools for handling prompts, managing memory, and connecting to external data sources, making it easier to build AI-driven workflows.\n\n2. **5*15**: The result of multiplying 5 by 15 is **75**.\n\n3. **Recent AI News Summary**:\n   - **Google\'s Gemini AI** introduced a "Projects" feature for organizing long-term research tasks, positioning AI as a productivity partner.\n   - **OpenAI\'s GPT-5** launched with improved reasoning but faced backlash for deprecating older models abruptly, causing user frustration.\n   - The **Trump administration** unveiled an AI Action Plan with 90+ policies, including relaxed AI chip export rules to China.\n   - **Meta** is investing in a "superintelligence lab" to develop AI rivaling human capabilities.\n   - **Anthropic** and startups like NeoLogic are advancing A